In [354]:
import bs4
import pandas
import numpy
import requests
from selenium import webdriver
import time
import random

In [355]:
company_name = input("Enter Company name : ")
location = input("Enter Location : ")

Enter Company name : airo digital labs
Enter Location : mumbai


In [356]:
url = f"https://www.naukri.com/{company_name}-jobs-in-{location}"

In [357]:
driver = webdriver.Chrome(executable_path='C:/program files/chromedriver.exe')

<ipython-input-357-7b27ac695236>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='C:/program files/chromedriver.exe')


In [358]:
Link=[]
driver.get(url+"-"+str(1))
timeDelay = random.randrange(10,20)
time.sleep(timeDelay) 
soup=bs4.BeautifulSoup(driver.page_source, 'lxml')#returns html of the page
for i in soup.findAll(attrs={'class':"jobTuple"}):
    for j in i.findAll(attrs={'class':"title ellipsis"}):
        Link.append(j.get('href')) #stores all the link of the job postings

In [359]:
len(Link)

20

In [360]:
description=[]
role=[]
skills=[]
links = []

for lin in range(len(Link)):
    driver.get(Link[lin])
    time.sleep(1)
    soup=bs4.BeautifulSoup(driver.page_source, 'lxml') 
    if soup.find(attrs={'class':"jd-header-comp-name"}) != None and company_name.lower() in soup.find(attrs={'class':"jd-header-comp-name"}).text.lower():
        description.append(soup.find(attrs={'class':"job-desc"}).text)
        for i in soup.find(attrs={'class':"other-details"}).findAll(attrs={'class':"details"}):
            role.append(i.text)
            break
        
        sk=[]
        for i in soup.find(attrs={'class':"key-skill"}).findAll('a'):
            sk.append(i.text)
        skills.append(sk)
        links.append(Link[lin])

In [361]:
df = pandas.DataFrame()

In [362]:
df['role']=role
df['description']=description
df['skills']=skills
df['Job Link']=links

In [363]:
df.head()

,role,description,skills,Job Link
0,"Role: Business Analyst,",Job description Business analyst responsibili...,"[Process automation, Business Analyst, ISO, An...",https://www.naukri.com/job-listings-business-a...
1,"Role: Full Stack Developer,",Job description Looking for a SCORM JS develo...,"[Process automation, Version control, GIT, Mac...",https://www.naukri.com/job-listings-xapi-devel...
2,"Role: UI / UX Designer,",Job description We are looking for a dynamic ...,"[Process automation, jQuery, Front end, Machin...",https://www.naukri.com/job-listings-ux-ui-desi...
3,"Role: Test Analyst,",Job description Quality Assurance Manual Data...,"[Process automation, QA, ERP, LMS, Testing too...",https://www.naukri.com/job-listings-quality-as...
4,"Role: Automation Test Engineer,",Job description A automation engineers design...,"[Maven, ERP, Coding, XML, MySQL, Debugging, Te...",https://www.naukri.com/job-listings-quality-as...


In [323]:
df.shape

(12, 4)

# Pre Processing NLP

In [324]:
# lower Casing 

df.description = df.description.apply(lambda x: x.lower())
df.head()

,role,description,skills,Job Link
0,"Role: Business Analyst,",job description business analyst responsibili...,"[Process automation, Business Analyst, ISO, An...",https://www.naukri.com/job-listings-business-a...
1,"Role: Full Stack Developer,",job description looking for a scorm js develo...,"[Process automation, Version control, GIT, Mac...",https://www.naukri.com/job-listings-xapi-devel...
2,"Role: UI / UX Designer,",job description we are looking for a dynamic ...,"[Process automation, jQuery, Front end, Machin...",https://www.naukri.com/job-listings-ux-ui-desi...
3,"Role: Test Analyst,",job description quality assurance manual data...,"[Process automation, QA, ERP, LMS, Testing too...",https://www.naukri.com/job-listings-quality-as...
4,"Role: Automation Test Engineer,",job description a automation engineers design...,"[Maven, ERP, Coding, XML, MySQL, Debugging, Te...",https://www.naukri.com/job-listings-quality-as...


In [325]:
# removing unwanted substrings from role
df['role'] = df['role'].str[6:-1]

In [326]:
import string
def remove_punctuation(text):
    for punc in list(string.punctuation):
        if punc in text:
            text = text.replace(punc, ' ')
    return text.strip()

In [327]:
 df["description"] = df["description"].apply(remove_punctuation)

In [328]:
from nltk import RegexpTokenizer

tokenizer = RegexpTokenizer(r"\w+") # Remove puntations
df['tokenized_sents'] = df.apply(lambda row: tokenizer.tokenize(row['description']), axis=1)

In [329]:
df['tokenized_sents'].head(10)

0    [job, description, business, analyst, responsi...
1    [job, description, looking, for, a, scorm, js,...
2    [job, description, we, are, looking, for, a, d...
3    [job, description, quality, assurance, manual,...
4    [job, description, a, automation, engineers, d...
5    [job, description, as, a, automation, test, le...
6    [job, description, we, are, looking, for, a, m...
7    [job, description, the, main, role, of, rpa, d...
8    [job, description, as, a, rpa, automation, eng...
9    [job, description, we, are, seeking, an, exper...
Name: tokenized_sents, dtype: object

In [330]:
# Stopward removal

from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
stop_words = [x for x in stop_words]

In [331]:
print(stop_words)

["you'll", 'with', 'out', 'yourself', 've', "shan't", 'just', 'by', 'will', "you'd", 'if', 'aren', 'once', 'having', 'here', "isn't", 'own', 'again', "couldn't", 'll', 'to', 'in', 'had', 'before', 'under', 'or', 'about', 'where', 'other', 'hadn', 'whom', 'same', 'yours', 'because', 'below', 'but', 'why', 'they', 'herself', 'have', 'you', 'yourselves', 'he', 'i', 'no', 'wasn', "aren't", 'him', 'further', 'ourselves', 'on', 'weren', 'that', "it's", 'its', 'very', 'me', 'them', "hasn't", 't', 'of', "wasn't", 'most', 'hers', 'we', "that'll", 's', 'not', 'needn', 'don', 'nor', 'up', 'doesn', 'only', 'there', "mustn't", 'itself', 'when', 'between', 'couldn', 'then', 'their', 'at', 'who', 'these', 'so', "you're", 'her', 'doing', 'his', 'for', 'm', "wouldn't", 'the', 'ours', 'such', 'mustn', 'am', 'both', "didn't", 'theirs', 'an', 'which', 'as', 'during', 'more', 'didn', 'haven', "needn't", 'over', "won't", 'from', 'mightn', 'can', 'any', 'this', "weren't", 'myself', 'be', 'my', "don't", 'won'

In [332]:
def remove_stopwords(x):
    st = []
    for i in x:
        if i in stop_words:
            continue
        else:
            st.append(i)
    return st

In [333]:
df['No_Stopwords'] = df['tokenized_sents'].apply(remove_stopwords)

In [334]:
df['No_Stopwords'].head(10)

0    [job, description, business, analyst, responsi...
1    [job, description, looking, scorm, js, develop...
2    [job, description, looking, dynamic, ui, ux, d...
3    [job, description, quality, assurance, manual,...
4    [job, description, automation, engineers, desi...
5    [job, description, automation, test, lead, arc...
6    [job, description, looking, machine, learning,...
7    [job, description, main, role, rpa, developers...
8    [job, description, rpa, automation, engineer, ...
9    [job, description, seeking, experienced, backe...
Name: No_Stopwords, dtype: object

In [335]:
# Lemmatization 

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [336]:
def lemmatize(x):
    lem_words =  []
    for ele in x:
        lem_words.append(lemmatizer.lemmatize(ele))
    return lem_words

In [337]:
df["Lemmatized"] = df["No_Stopwords"].apply(lemmatize)

In [338]:
df["Lemmatized"].head()

0    [job, description, business, analyst, responsi...
1    [job, description, looking, scorm, j, develope...
2    [job, description, looking, dynamic, ui, ux, d...
3    [job, description, quality, assurance, manual,...
4    [job, description, automation, engineer, desig...
Name: Lemmatized, dtype: object

In [339]:
# pos tagging

from nltk import pos_tag

In [340]:
df["POS_Lemmataized"] = df["Lemmatized"].apply(pos_tag)

In [341]:
df["POS_Lemmataized"].head()

0    [(job, NN), (description, NN), (business, NN),...
1    [(job, NN), (description, NN), (looking, VBG),...
2    [(job, NN), (description, NN), (looking, VBG),...
3    [(job, NN), (description, NN), (quality, NN), ...
4    [(job, NN), (description, NN), (automation, NN...
Name: POS_Lemmataized, dtype: object

In [342]:
def select_NN(x):
    task_keywords = []
    for i in x:
        if i[1] == "NN":
            task_keywords.append(i[0])
        else:
            continue
    return task_keywords

In [343]:
df['KeyWords'] = df['POS_Lemmataized'].apply(select_NN)

In [344]:
df['KeyWords']

0     [job, description, business, analyst, responsi...
1     [job, description, j, developer, work, team, w...
2     [job, description, designer, experience, user,...
3     [job, description, quality, assurance, databas...
4     [job, description, automation, engineer, desig...
5     [job, description, automation, test, lead, own...
6     [job, description, machine, intelligence, prod...
7     [job, description, role, rpa, developer, build...
8     [job, description, rpa, automation, lead, solu...
9     [job, description, backend, developer, buildin...
10    [job, description, pl, sql, developer, network...
11    [job, description, test, functionality, usabil...
Name: KeyWords, dtype: object

In [345]:
import pickle

with open('vocab.pkl', 'rb') as file:
    vocab = pickle.load(file)
    print(vocab)

['model', 'machine', 'nlp', 'machine learning', 'artificial intelligence', 'artificial intelligence', 'data collection', 'data scientist', 'tensorflow', 'data science', 'natural language processing', 'aiml', 'ai algorithms', 'pandas', 'numpy', 'pytorch', 'ml', 'predictive modelling']


In [346]:
df.head()

,role,description,skills,Job Link,tokenized_sents,No_Stopwords,Lemmatized,POS_Lemmataized,KeyWords
0,Business Analyst,job description business analyst responsibili...,"[Process automation, Business Analyst, ISO, An...",https://www.naukri.com/job-listings-business-a...,"[job, description, business, analyst, responsi...","[job, description, business, analyst, responsi...","[job, description, business, analyst, responsi...","[(job, NN), (description, NN), (business, NN),...","[job, description, business, analyst, responsi..."
1,Full Stack Developer,job description looking for a scorm js develo...,"[Process automation, Version control, GIT, Mac...",https://www.naukri.com/job-listings-xapi-devel...,"[job, description, looking, for, a, scorm, js,...","[job, description, looking, scorm, js, develop...","[job, description, looking, scorm, j, develope...","[(job, NN), (description, NN), (looking, VBG),...","[job, description, j, developer, work, team, w..."
2,UI / UX Designer,job description we are looking for a dynamic ...,"[Process automation, jQuery, Front end, Machin...",https://www.naukri.com/job-listings-ux-ui-desi...,"[job, description, we, are, looking, for, a, d...","[job, description, looking, dynamic, ui, ux, d...","[job, description, looking, dynamic, ui, ux, d...","[(job, NN), (description, NN), (looking, VBG),...","[job, description, designer, experience, user,..."
3,Test Analyst,job description quality assurance manual data...,"[Process automation, QA, ERP, LMS, Testing too...",https://www.naukri.com/job-listings-quality-as...,"[job, description, quality, assurance, manual,...","[job, description, quality, assurance, manual,...","[job, description, quality, assurance, manual,...","[(job, NN), (description, NN), (quality, NN), ...","[job, description, quality, assurance, databas..."
4,Automation Test Engineer,job description a automation engineers design...,"[Maven, ERP, Coding, XML, MySQL, Debugging, Te...",https://www.naukri.com/job-listings-quality-as...,"[job, description, a, automation, engineers, d...","[job, description, automation, engineers, desi...","[job, description, automation, engineer, desig...","[(job, NN), (description, NN), (automation, NN...","[job, description, automation, engineer, desig..."


In [347]:
def check_ML_Job(x):
    temp_role = x[0].lower().split()
    
     
    # checking job title
    for word in temp_role:
        if "machine learning" == word or "ml" == word:
            return "Yes"
        
    # Checking Description  
    temp_desc = x[-1]
    for v in vocab:
        if v in temp_desc:
            print("-----------------------",temp_desc,"------------------------------------")
            return "Yes"
    return "False"

In [348]:
df["ML Job"] = df.apply(check_ML_Job,axis=1)

----------------------- ['job', 'description', 'machine', 'intelligence', 'product', 'machine', 'responsibility', 'machine', 'model', 'system', 'job', 'skill', 'programming', 'goal', 'shape', 'self', 'application', 'duty', 'responsibility', 'build', 'algorithm', 'system', 'end', 'solution', 'batch', 'time', 'algorithm', 'performance', 'b', 'spirit', 'opportunity', 'business', 'process', 'consumer', 'experience', 'prototype', 'solution', 'demonstrate', 'value', 'crawl', 'work', 'scientist', 'analyst', 'product', 'feature', 'process', 'analysis', 'model', 'development', 'validation', 'implementation', 'efficient', 'software', 'product', 'release', 'environment', 'promote', 'software', 'engineering', 'practice', 'team', 'knowledge', 'team', 'adopt', 'practice', 'use', 'community', 'practice', 'education', 'bachelor', 'computer', 'science', 'role', 'stack', 'developer', 'industry', 'type', 'courier', 'department', 'engineering', 'software', 'employment', 'type', 'time', 'category', 'softwa

In [349]:
df[df["ML Job"]=="Yes"]

,role,description,skills,Job Link,tokenized_sents,No_Stopwords,Lemmatized,POS_Lemmataized,KeyWords,ML Job
6,Full Stack Developer,job description we are looking for a machine ...,"[Prototype, Artificial Intelligence, Machine l...",https://www.naukri.com/job-listings-ai-ml-deve...,"[job, description, we, are, looking, for, a, m...","[job, description, looking, machine, learning,...","[job, description, looking, machine, learning,...","[(job, NN), (description, NN), (looking, VBG),...","[job, description, machine, intelligence, prod...",Yes


In [350]:
ML_JOBS = df[df["ML Job"] == "Yes"].values.tolist()
result = []
for job in ML_JOBS:
    result.append([job[0],job[3]])
result

[['Full Stack Developer',
  'https://www.naukri.com/job-listings-ai-ml-developer-airo-digital-labs-india-pvt-ltd-kolkata-mumbai-new-delhi-hyderabad-secunderabad-pune-chennai-bangalore-bengaluru-4-to-9-years-180423500502']]

In [351]:
len(result)

1

In [352]:
def unique(result):
    unique_list = []
    for x in result:
        if x[1] not in unique_list:
            unique_list.append(x)
    for x in unique_list:
        print(x)
    print(len(unique_list))

In [353]:
unique(result)

['Full Stack Developer', 'https://www.naukri.com/job-listings-ai-ml-developer-airo-digital-labs-india-pvt-ltd-kolkata-mumbai-new-delhi-hyderabad-secunderabad-pune-chennai-bangalore-bengaluru-4-to-9-years-180423500502']
1
